## Join the Duet Server the Data Owner 1 connected to

In [1]:
import syft as sy
duet1 = sy.join_duet(loopback=True,network_url="http://localhost:5000/")

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://localhost:5000/
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!


/home/holly/anaconda3/envs/pysyft/lib/python3.9/site-packages/aiortc/rtcdtlstransport.py:211: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  _openssl_assert(lib.SSL_CTX_use_certificate(ctx, self._cert._x509) == 1)  # type: ignore
/home/holly/anaconda3/envs/pysyft/lib/python3.9/site-packages/aiortc/rtcdtlstransport.py:186: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  value=certificate_digest(self._cert._x509),  # type: ignore



♫♫♫ > CONNECTED!


## Join the Duet Server the Data Owner 2 connected to

In [2]:
duet2 = sy.join_duet(loopback=True,network_url="http://localhost:5000/")

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://localhost:5000/
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > CONNECTED!


In [3]:
duet1.store.pandas

,ID,Tags,Description,object_type
0,<UID: 2e8eea1659ee4107b2effa8442b71063>,[breast_cancer-data],这是威斯康星洲临床科学中心开元的乳腺癌肿瘤数据集，共30维的特征数据,<class 'torch.Tensor'>
1,<UID: 73731e7e73834f598f26e922cb58040d>,[breast_cancer-target],这是数据标签，1代表良性肿瘤，0代表恶性肿瘤,<class 'torch.Tensor'>


In [4]:
duet2.store.pandas

,ID,Tags,Description,object_type
0,<UID: e558f4567b8a46c9afdc1c5c23068509>,[breast_cancer-data],这是威斯康星洲临床科学中心开元的乳腺癌肿瘤数据集，共30维的特征数据,<class 'torch.Tensor'>
1,<UID: 6e3469d501164615a4e783a72b2c0ddc>,[breast_cancer-target],这是数据标签，1代表良性肿瘤，0代表恶性肿瘤,<class 'torch.Tensor'>


## Linear regression

In [5]:
data_ptr1 = duet1.store[0]
target_ptr1 = duet1.store[1]
data_ptr2 = duet2.store[0]
target_ptr2 = duet2.store[1]

In [6]:
print(data_ptr1)
print(target_ptr1)
print(data_ptr2)
print(target_ptr2)

In [7]:
print(duet1.store.pandas["Description"][0])
print()
print(duet1.store.pandas["Description"][1])

这是威斯康星洲临床科学中心开元的乳腺癌肿瘤数据集，共30维的特征数据

这是数据标签，1代表良性肿瘤，0代表恶性肿瘤


### Create Base Model

In [8]:
import torch

In [9]:
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
        self.fc1 = self.torch_ref.nn.Linear(30, 10)
        self.fc2 = self.torch_ref.nn.Linear(10, 1)
        self.sigmoid=self.torch_ref.nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.sigmoid(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

In [10]:
local_model = SyNet(torch)

将模型发动给dataowner1和dataowner2

In [11]:
remote_model1 = local_model.send(duet1)

In [12]:
remote_model2 = local_model.send(duet2)

远程的torch

In [13]:
remote_torch1 = duet1.torch

In [14]:
remote_torch2 = duet2.torch

设置优化器

In [16]:
params1 = remote_model1.parameters()
optim1 = remote_torch1.optim.Adam(params=params1, lr=0.01)
params2 = remote_model2.parameters()
optim2 = remote_torch2.optim.Adam(params=params2, lr=0.01)

In [37]:
async def train(ownername,iterations, model, torch_ref, optim, data_ptr, target_ptr):

    losses = []
    #损失函数
    cost=torch_ref.nn.BCELoss()
    for i in range(iterations):
        
        #预测结果
        output = model(data_ptr)
        
        #计算损失值
        loss=cost(output,target_ptr)
        
        #在反向传播前先把梯度清零
        optim.zero_grad()
        
        #反向传播,计算各参数对于损失loss的梯度
        loss.backward()
        
        #根据刚刚反向传播得到的梯度更新模型参数
        optim.step()        

        loss_item = loss.item()
        
        loss_value = loss_item.get(
            reason="To evaluate training progress", request_block=True, timeout_secs=5,delete_obj=False)

        if i % 100 == 0:
            print(ownername,"--Epoch", i, "loss", loss_value)

        losses.append(loss_value)
        
    #返回模型
    latest_model=model.get(
        request_block=True,
        reason="To run test and inference locally",
        timeout_secs=5,
    )

    return latest_model

设置迭代次数

In [38]:
iteration = 1

In [39]:
import asyncio
results = await asyncio.gather(
        *[
            train("dataowner1",iteration, remote_model1, remote_torch1, optim1, data_ptr1, target_ptr1),
            train("dataowner2",iteration, remote_model2, remote_torch2, optim2, data_ptr2, target_ptr2)
        ]
    )

dataowner2 --Epoch 0 loss 0.6857185363769531
dataowner1 --Epoch 0 loss 0.6857185363769531


In [43]:
print(results)

[<__main__.SyNet object at 0x7f920d22f160>, <__main__.SyNet object at 0x7f920d22fb50>]


## FedAvg

### Training

#### Send one copy of the model to each data owner or client and train them remotely one by one

Dummy Target data

In [ ]:
target2_ptr = th.FloatTensor(np.array([35, 40, 45, 55, 60]).reshape(-1, 1))
target2_ptr

In [ ]:
iteration = 10
losses = train(iteration, remote_model2, remote_torch2, optim2, data2_ptr, target2_ptr)

### Averaging Model Updates

Ideally, there will be a coordinator server with a secure aggreagtor who will get the model updates from different clients and make an aggregation. For the case of simplicity, in this example we will make the Data Sceintist server work as the coordinator.

### Little sanity check!

In [ ]:
param1 = remote_model1.parameters().get(request_block=True,delete_obj=False)
param2 = remote_model2.parameters().get(request_block=True,delete_obj=False)

print("Base Model parameters:")
print(base_model.parameters())
print()

print("Remote model1 parameters:")
print(param1)
print()

print("Remote model2 parameters:")
print(param2)

In [ ]:
print(remote_model1)

As you can see, the remote model paramter values are different from the base model paramter values. That means the remote copies of our base model got trained and updated.

In [ ]:
remote_model1_updates = remote_model1.get(
    request_block=True
).state_dict()

print(remote_model1_updates)

In [ ]:
remote_model2_updates = remote_model2.get(
    request_block=True
).state_dict()

print(remote_model2_updates)

In [ ]:
from collections import OrderedDict

Let's do the aggregation of the weights. In this example, we will just calculate the average of corresponding weights from each model.

In [ ]:
avg_updates = OrderedDict()
avg_updates["linear.weight"] = (
    remote_model1_updates["linear.weight"] + remote_model2_updates["linear.weight"]
) / 2
avg_updates["linear.bias"] = (
    remote_model1_updates["linear.bias"] + remote_model2_updates["linear.bias"]
) / 2

print(avg_updates)

### Load aggregated weights

In [ ]:
combined_model = SyNet(torch)

In [ ]:
combined_model.load_state_dict(avg_updates)

In [ ]:
del avg_updates, remote_model1_updates, remote_model2_updates

In [ ]:
test_data = th.FloatTensor(np.array([17, 25, 32, 50, 80]).reshape(-1, 1))
test_target = th.FloatTensor(np.array([12, 15, 20, 30, 50]).reshape(-1, 1))

In [ ]:
preds = []
with torch.no_grad():
    for i in range(len(test_data)):
        sample = test_data[i]
        y_hat = combined_model(sample)

        print(f"Prediction: {y_hat.item()} Ground Truth: {test_target[i].item()}")
        preds.append(y_hat)

In [ ]:
preds = []
with torch.no_grad():
    for i in range(len(test_data)):
        sample = test_data[i]
        y_hat = base_model(sample)

        print(f"Prediction: {y_hat.item()} Ground Truth: {test_target[i].item()}")
        preds.append(y_hat)

## Comparison to classical linear regression on centralised data

In [ ]:
import torch
import numpy as np

in_dim = 1
out_dim = 1


class ClassicalLR(torch.nn.Module):
    def __init__(self, torch):
        super(ClassicalLR, self).__init__()
        self.linear = torch.nn.Linear(in_dim, out_dim)

    def forward(self, x):
        x = self.linear(x)
        return x


classical_model = ClassicalLR(torch)

In [ ]:
data = torch.FloatTensor(
    np.array([5, 15, 25, 35, 45, 55, 60, 65, 75, 85, 95]).reshape(-1, 1)
)
target = torch.FloatTensor(
    np.array([5, 10, 15, 22, 30, 38, 35, 40, 45, 55, 60]).reshape(-1, 1)
)

In [ ]:
def classic_train(iterations, model, torch, optim, data, target, criterion):

    losses = []

    for i in range(iterations):

        optim.zero_grad()

        output = model(data)

        loss = criterion(output, target)

        loss_item = loss.item()

        if i % 10 == 0:
            print("Epoch", i, "loss", loss_item)

        losses.append(loss_item)

        loss.backward()

        optim.step()

    return losses

In [ ]:
params = classical_model.parameters()
optim = torch.optim.Adam(params=params, lr=0.1)
criterion = torch.nn.MSELoss()

In [ ]:
iteration = 100
losses = classic_train(
    iteration, classical_model, torch, optim, data, target, criterion
)

In [ ]:
test_data = th.FloatTensor(np.array([17, 25, 32, 50, 80]).reshape(-1, 1))
test_target = th.FloatTensor(np.array([12, 15, 20, 30, 50]).reshape(-1, 1))

In [ ]:
preds = []
with torch.no_grad():
    for i in range(len(test_data)):
        sample = test_data[i]
        y_hat = classical_model(sample)

        print(f"Prediction: {y_hat.item()} Ground Truth: {test_target[i].item()}")
        preds.append(y_hat)